In [14]:
!pip install sentence_transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached transformers-4.23.1-py3-none-any.whl (5.3 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     ------------------------------------- 161.9/161.9 MB 29.8 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 70.4 MB/s eta 0:00:00
     ---------------------------------------- 7.3/7.3 MB 93.9 MB/s eta 0:00:00
     --------------------------------------- 39.8/39.8 MB 65.5 MB/s eta 0:00:00
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
     ---------------------------------------- 1.1/1.1 MB 70.2 MB/s eta 0:00:00
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
     ---------------------------------------- 155.4/155.4 kB ? eta 0:00:00
  Using cached filelock-3.8.0-py3-none-any.whl (10 kB)
     ---------------------------------------- 3.3/3.3 MB 69.8 MB/s eta 0:00:00
   

In [2]:
!pip install konlpy

In [19]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from konlpy.tag import Mecab
from konlpy.tag import Komoran
from collections import Counter

In [20]:
mecab = Mecab(dicpath=r'C:\mecab\mecab-ko-dic')
out = mecab.pos(phrase = "오늘은 맑은 날씨이다.")
[word[0] for word in out if word[1] == 'NNG' or word[1] == 'NNP']

['오늘', '날씨']

In [134]:
doc = """
"박홍근 위원] \"그러면 지금 마지막 남은 여섯 번째 감시시편 있지 않습니까? 이 부분은 언제 꺼내서 할 계획으로 돼 있습니까?\"\n한국원자력안전기술원장 박윤원] \"그것은 저희가 꺼내서 하는 것은 아니고요. 아까 아마 그 지적하신 것들이 말씀하신 것들이 ‘2017년 이후에 만약에 목표를 한다면 그것은 연장을 목표로 하는 것 아니냐’하고 말씀하셨지 않습니까? 그래서 아마 미국에서 저희가 그 규정을 미국 것을 그냥 따 와서 하다 보니까 그렇게 됐는데 그것 자체가 보면 미국 자체가 60～80년 이렇게까지를 감안을 해 가지고 고려를 하고 있는 겁니다.   그런데 우리나라에서도 사실은 보면 2017년까지에 해당되는 건전성은 이미 평가가 끝났고 그 이후에 만약에 필요하다면 그러니까 계속운전을 하느냐 마나냐는 2017년 이전 최소한 2～5년 전에 결정이 돼야 되겠지요. 그것은 또 한수원이 결정해야 될 문제라고 봅니다마는 그렇게 해서 만약에 간다면 그때 그 시편을 또 사용할 수 있을 거라고 생각은 합니다.\"\n박홍근 위원] \"그러니까 아직은 구체적인 시기가 결정된 바는 없다 이거지요?\"\n한국원자력안전기술원장 박윤원] \"아직은 없습니다.\"\n박홍근 위원] \"그러니까 2017년 이전에 2～3년 전에 기본적으로 해야 한다 이런 말씀이시잖아요?   그런데 말씀드리는 것은 이렇습니다. 2005년도의 이 결과가 2013년도에는 기준을 일단은 지금 초과하는 것으로 나타났잖아요? 그러면 2013년과 말씀하신 그 특정 이후의 추출 꺼내는 시점 있지 않습니까? 그 공백의 사이에 이 용기 안에서의 그런 어떤 문제점에 대해서는 누가 그것을 책임질 수 있나요?\"\n",
"""

In [136]:
out = mecab.pos(doc)
noun_list = [word[0] for word in out if word[1] == 'NNG' or word[1] == 'NNP']
noun_list = [noun for noun in noun_list if len(noun) > 1]

In [17]:
# okt = Okt()

# tokenized_doc = okt.pos(doc, stem=True)
# tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

In [137]:
stop_words = ['위원장', '위원', '선배', '저', '우리', '위', '가지', '안녕하십니까', '여러', '위해', '못', '것', '사실', '말씀', '이후', '만약', '원장', '이전', '마지막', '최소한', '생각', '사이']
stop_names = ['강창순', '유기홍', '박성호', '유성엽', '유은혜', '정진후', '박홍근', '박윤원']

stop_list = stop_words + stop_names

In [138]:
result = []
for w in noun_list:
    if w not in stop_list:
        result.append(w)
        
tokenized_nouns = ' '.join(result)

# 중복제거
unique_token_koms_noun = set(result)
unique_noun_list = ' '.join(unique_token_koms_noun)

# 명사 빈도 카운트
count = Counter(result)

common_list = count.most_common(10)

In [139]:
tokenized_nouns = unique_noun_list
tokenized_nouns

'안전 초과 미국 감안 사용 고려 문제 나라 원자력 해당 시기 기술 공백 규정 평가 문제점 계획 한국 시편 목표 지적 수원 기준 기본 자체 추출 그때 필요 구체 연장 결정 시점 결과 부분 운전 특정 용기 감시'

In [140]:
common_list

[('미국', 3),
 ('결정', 3),
 ('시편', 2),
 ('한국', 2),
 ('원자력', 2),
 ('안전', 2),
 ('기술', 2),
 ('목표', 2),
 ('자체', 2),
 ('감시', 1)]

In [141]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 73
trigram 다섯개만 출력 : ['감안 사용' '감안 사용 고려' '결과 부분' '결과 부분 운전' '결정 시점']


In [142]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [143]:
# SBERT로 키워드를 수치화
#model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

In [144]:
doc_embedding = model.encode([doc])

In [145]:
candidate_embeddings = model.encode(candidates)

In [146]:
# 가장 유사한 키워드를 추출합니다
# 상위 5개
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['결과 부분 운전', '미국 감안', '문제점 계획 한국', '초과 미국 감안', '계획 한국']


In [27]:
# 비슷한 키워드가 리턴되는 이유는 문서를 가장 잘 나타내고 있으므로
# 따라서 좀 다양한 키워드를 원한다면 아래의 알고리즘 사용

In [147]:
# Max Sum Similarity
# 유사성을 최소화하면서 문서와의 유사성은 극대화

def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [148]:
# nr_candidates가 낮으면 유사한 키워드들
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['한국 시편 목표', '문제 나라 원자력', '부분 운전', '초과 미국 감안', '계획 한국']

In [149]:
# nr_candidates가 높으면 다양한 키워드들 (30 넘어가면 시간이 너무 오래 걸림)
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['해당 시기', '문제점 계획', '결과 부분', '기본 자체', '한국 시편 목표']

In [150]:
# Maximal Marginal Relevance 알고리즘
# 중복을 최소화하고 결과의 다양성을 극대화

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [151]:
# diversity가 낮으면 유사한 단어들
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['계획 한국', '결과 부분 운전', '문제점 계획 한국', '초과 미국 감안', '고려 문제 나라']

In [152]:
# diversity가 높으면 다양한 키워드
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['계획 한국', '공백 규정 평가', '필요 구체', '수원 기준 기본', '결과 부분 운전']

In [46]:
!pip install keybert

  Using cached keybert-0.6.0-py2.py3-none-any.whl (22 kB)
  Using cached rich-12.6.0-py3-none-any.whl (237 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)


In [154]:
from transformers import BertModel
from keybert import KeyBERT

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [155]:
doc2 = """
"박홍근 위원] \"그러면 지금 마지막 남은 여섯 번째 감시시편 있지 않습니까? 이 부분은 언제 꺼내서 할 계획으로 돼 있습니까?\"\n한국원자력안전기술원장 박윤원] \"그것은 저희가 꺼내서 하는 것은 아니고요. 아까 아마 그 지적하신 것들이 말씀하신 것들이 ‘2017년 이후에 만약에 목표를 한다면 그것은 연장을 목표로 하는 것 아니냐’하고 말씀하셨지 않습니까? 그래서 아마 미국에서 저희가 그 규정을 미국 것을 그냥 따 와서 하다 보니까 그렇게 됐는데 그것 자체가 보면 미국 자체가 60～80년 이렇게까지를 감안을 해 가지고 고려를 하고 있는 겁니다.   그런데 우리나라에서도 사실은 보면 2017년까지에 해당되는 건전성은 이미 평가가 끝났고 그 이후에 만약에 필요하다면 그러니까 계속운전을 하느냐 마나냐는 2017년 이전 최소한 2～5년 전에 결정이 돼야 되겠지요. 그것은 또 한수원이 결정해야 될 문제라고 봅니다마는 그렇게 해서 만약에 간다면 그때 그 시편을 또 사용할 수 있을 거라고 생각은 합니다.\"\n박홍근 위원] \"그러니까 아직은 구체적인 시기가 결정된 바는 없다 이거지요?\"\n한국원자력안전기술원장 박윤원] \"아직은 없습니다.\"\n박홍근 위원] \"그러니까 2017년 이전에 2～3년 전에 기본적으로 해야 한다 이런 말씀이시잖아요?   그런데 말씀드리는 것은 이렇습니다. 2005년도의 이 결과가 2013년도에는 기준을 일단은 지금 초과하는 것으로 나타났잖아요? 그러면 2013년과 말씀하신 그 특정 이후의 추출 꺼내는 시점 있지 않습니까? 그 공백의 사이에 이 용기 안에서의 그런 어떤 문제점에 대해서는 누가 그것을 책임질 수 있나요?\"\n",
      """

In [156]:
doc2

'\n"박홍근 위원] "그러면 지금 마지막 남은 여섯 번째 감시시편 있지 않습니까? 이 부분은 언제 꺼내서 할 계획으로 돼 있습니까?"\n한국원자력안전기술원장 박윤원] "그것은 저희가 꺼내서 하는 것은 아니고요. 아까 아마 그 지적하신 것들이 말씀하신 것들이 ‘2017년 이후에 만약에 목표를 한다면 그것은 연장을 목표로 하는 것 아니냐’하고 말씀하셨지 않습니까? 그래서 아마 미국에서 저희가 그 규정을 미국 것을 그냥 따 와서 하다 보니까 그렇게 됐는데 그것 자체가 보면 미국 자체가 60～80년 이렇게까지를 감안을 해 가지고 고려를 하고 있는 겁니다.   그런데 우리나라에서도 사실은 보면 2017년까지에 해당되는 건전성은 이미 평가가 끝났고 그 이후에 만약에 필요하다면 그러니까 계속운전을 하느냐 마나냐는 2017년 이전 최소한 2～5년 전에 결정이 돼야 되겠지요. 그것은 또 한수원이 결정해야 될 문제라고 봅니다마는 그렇게 해서 만약에 간다면 그때 그 시편을 또 사용할 수 있을 거라고 생각은 합니다."\n박홍근 위원] "그러니까 아직은 구체적인 시기가 결정된 바는 없다 이거지요?"\n한국원자력안전기술원장 박윤원] "아직은 없습니다."\n박홍근 위원] "그러니까 2017년 이전에 2～3년 전에 기본적으로 해야 한다 이런 말씀이시잖아요?   그런데 말씀드리는 것은 이렇습니다. 2005년도의 이 결과가 2013년도에는 기준을 일단은 지금 초과하는 것으로 나타났잖아요? 그러면 2013년과 말씀하신 그 특정 이후의 추출 꺼내는 시점 있지 않습니까? 그 공백의 사이에 이 용기 안에서의 그런 어떤 문제점에 대해서는 누가 그것을 책임질 수 있나요?"\n",\n      '

In [157]:
out = mecab.pos(doc2)
noun_list = [word[0] for word in out if word[1] == 'NNG' or word[1] == 'NNP']
noun_list = [noun for noun in noun_list if len(noun) > 1]
noun_list

['박홍근',
 '위원',
 '마지막',
 '감시',
 '시편',
 '부분',
 '계획',
 '한국',
 '원자력',
 '안전',
 '기술',
 '원장',
 '박윤원',
 '지적',
 '말씀',
 '이후',
 '만약',
 '목표',
 '연장',
 '목표',
 '말씀',
 '미국',
 '규정',
 '미국',
 '자체',
 '미국',
 '자체',
 '감안',
 '고려',
 '나라',
 '해당',
 '평가',
 '이후',
 '만약',
 '필요',
 '운전',
 '이전',
 '최소한',
 '결정',
 '수원',
 '결정',
 '문제',
 '만약',
 '그때',
 '시편',
 '사용',
 '생각',
 '박홍근',
 '위원',
 '구체',
 '시기',
 '결정',
 '한국',
 '원자력',
 '안전',
 '기술',
 '원장',
 '박윤원',
 '박홍근',
 '위원',
 '이전',
 '기본',
 '말씀',
 '말씀',
 '결과',
 '기준',
 '초과',
 '말씀',
 '특정',
 '이후',
 '추출',
 '시점',
 '공백',
 '사이',
 '용기',
 '문제점']

In [158]:
result = []
for w in noun_list:
    if w not in stop_list:
        result.append(w)
        
tokenized_nouns = ' '.join(result)

# 중복제거
unique_token_koms_noun = set(result)
unique_noun_list = ' '.join(unique_token_koms_noun)

# 명사 빈도 카운트
count = Counter(result)

common_list = count.most_common(10)

In [159]:
common_list

[('미국', 3),
 ('결정', 3),
 ('시편', 2),
 ('한국', 2),
 ('원자력', 2),
 ('안전', 2),
 ('기술', 2),
 ('목표', 2),
 ('자체', 2),
 ('감시', 1)]

In [160]:
tokenized_nouns = unique_noun_list
tokenized_nouns

'안전 초과 미국 감안 사용 고려 문제 나라 원자력 해당 시기 기술 공백 규정 평가 문제점 계획 한국 시편 목표 지적 수원 기준 기본 자체 추출 그때 필요 구체 연장 결정 시점 결과 부분 운전 특정 용기 감시'

In [161]:
kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1, 1), stop_words=None)

[('원자력', 0.5887),
 ('안전', 0.5808),
 ('감시', 0.5639),
 ('규정', 0.4851),
 ('한국', 0.4628)]

In [162]:
kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(2, 2), stop_words=None)

[('나라 원자력', 0.6155),
 ('안전 초과', 0.6067),
 ('원자력 해당', 0.594),
 ('기술 공백', 0.5337),
 ('규정 평가', 0.5295)]

In [163]:
kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1,2), stop_words='english',
                              use_maxsum=True, nr_candidates=30, top_n=5)

[('운전 특정', 0.4178),
 ('한국', 0.4628),
 ('규정 평가', 0.5295),
 ('기술 공백', 0.5337),
 ('나라 원자력', 0.6155)]

In [164]:
# 다양
kw_model.extract_keywords(tokenized_nouns, keyphrase_ngram_range=(1,2), stop_words='english',
                              use_mmr=True, diversity=0.7)

[('나라 원자력', 0.6155),
 ('기술 공백', 0.5337),
 ('규정 평가', 0.5295),
 ('한국 시편', 0.3419),
 ('운전', 0.2757)]